In [1]:
import pickle

from utils import *
import pandas as pd 

## Load Radicals 

199 rows × 3 columns

In [2]:
df_rad_1 = pd.read_csv("../resources/radicals_utf8.csv")

In [3]:
df_rad_1

,zi,is_radical,is_zi
0,八,1,1
1,白,1,1
2,勹,1,0
3,贝,1,1
4,鼻,1,1
...,...,...,...
194,隹,1,0
195,子,1,1
196,自,1,1
197,走,1,1


In [5]:
with DBConn(db_file="zinets.sqlite") as _conn:
    df_rad_1.to_sql("t_radical_1", con=_conn, if_exists="append", index=False)

In [10]:
df_rad_1.columns 

Index(['zi', 'is_radical', 'is_zi'], dtype='object')

## Load CEdict (all)

13564 rows × 10 columns

In [8]:
df_cedict_all = pd.read_csv("../resources/zi_cedict.csv").fillna("")

In [9]:
df_cedict_all

,zi,desc,pinyin,nstrokes,alias,is_radical,is_zi,is_traditional,desc_en,is_active
0,A,,A,,A,0,1,0,(slang) (Tw) to steal,1
1,阿,,A1,,阿,0,1,0,abbr. for Afghanistan 阿富汗[A1 fu4 han4],1
2,哀,,Ai1,,哀,0,1,0,"Ai (c. 2000 BC), sixth of legendary Flame Empe...",1
3,埃,,Ai1,,埃,0,1,0,abbr. for Egypt 埃及[Ai1 ji2],1
4,毐,,Ai3,,毐,0,1,0,see historical character Lao Ai 嫪毐/by extensio...,1
...,...,...,...,...,...,...,...,...,...,...
13559,祚,,zuo4,,祚,0,1,0,blessing/the throne,1
13560,胙,,zuo4,,胙,0,1,0,to grant or bestow/sacrificial flesh offered t...,1
13561,葄,,zuo4,,葄,0,1,0,straw cushion/pillow,1
13562,酢,,zuo4,,酢,0,1,0,toast to host by guest,1


In [11]:
with DBConn(db_file="zinets.sqlite") as _conn:
    df_cedict_all.to_sql("t_cedict_all", con=_conn, if_exists="append", index=False)

In [12]:
df_cedict_all.columns 

Index(['zi', 'desc', 'pinyin', 'nstrokes', 'alias', 'is_radical', 'is_zi',
       'is_traditional', 'desc_en', 'is_active'],
      dtype='object')

Do not use `t_cedict_all` table, use `t_zi_cedict` (with id) instead

## Load CEdict (deduped)

4605 rows × 7 columns

In [15]:
df_cedict_dedup = pd.read_csv("../resources/zi_cedict-dup-deduped-2023-09-16.csv", sep="\t")

In [16]:
df_cedict_dedup

,id,zi,alias,pinyin,is_radical,is_active,desc_en
0,1298,㝵,㝵,ai4,0,0,variant of 礙|碍[ai4]
1,2895,㝵,㝵,de2,0,0,to obtain (old variant of 得[de2])
2,9198,㲈,㲈,shao2,0,0,variant of 韶[shao2]
3,9944,㲈,㲈,tao2; shao2,0,1,variant of 鞀|鼗[tao2]
4,8396,䕭,䕭,qian2,0,0,variant of 蕁|荨[qian2]
...,...,...,...,...,...,...,...
4600,4411,𬮤,閤,he2,0,1,variant of 闔|阖[he2] smooth/sly slippery not...
4601,1141,𬮱,闉,yin1,0,0,surname Yin
4602,12004,𬮱,闉,yin1,0,1,inner gates
4603,1132,𬺈,齮,yi3,0,0,surname Yi


In [17]:
with DBConn(db_file="zinets.sqlite") as _conn:
    df_cedict_dedup.to_sql("t_cedict_dedup", con=_conn, if_exists="append", index=False)

In [18]:
df_cedict_dedup.columns 

Index(['id', 'zi', 'alias', 'pinyin', 'is_radical', 'is_active', 'desc_en'], dtype='object')


```
select is_active, count(*) from t_cedict_dedup group by is_active;
0	2011
1	2594

```

populate t_zi from 

## Populate t_zi

t_zi_cedict - t_cedict_dedup + t_cedict_dedup (is_active)

In [19]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        select id, zi, pinyin, alias, desc_en, is_active
        from t_zi_cedict c
        where not exists (
            select 1 
            from t_cedict_dedup d
            where c.id = d.id
        )
        
    """
    df_1 = pd.read_sql(sql_stmt, _conn)

In [20]:
df_1   # 8922 rows × 6 columns

,id,zi,pinyin,alias,desc_en,is_active
0,1,雏,chu2,雛,(bound form) chick; young bird,1
1,2,仍,reng2,仍,still; yet/to remain/(literary) frequently,1
2,3,但,dan4,但,but; yet,1
3,4,脩,xiu1,脩,dried meat presented by pupils to their teache...,1
4,5,智,zhi4,智,(literary) wise; wisdom,1
...,...,...,...,...,...,...
8917,13519,葄,zuo4,葄,straw cushion/pillow,1
8918,13521,阼,zuo4,阼,steps leading to the eastern door,1
8919,13522,毐,ai3,毐,see historical character Lao Ai 嫪毐/by extensio...,0
8920,13524,豳,bin1,豳,name of an ancient county in Shaanxi/variant o...,0


In [21]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        select id, zi, pinyin, alias, desc_en, is_active
        from t_cedict_dedup d
        where is_active = 1
    """
    df_2 = pd.read_sql(sql_stmt, _conn)

In [22]:
df_2   # 2594 rows × 6 columns

,id,zi,pinyin,alias,desc_en,is_active
0,9944,㲈,tao2; shao2,㲈,variant of 鞀|鼗[tao2],1
1,3094,丁,ding1,丁,"""male adult/the 4th of the 10 Heavenly Stems 天...",1
2,10384,万,wan4,萬,ten thousand/a great number,1
3,9019,三,san1,三,three/3,1
4,11,上,shang4,上,(bound form) up; upper,1
...,...,...,...,...,...,...
2589,8795,𬒈,que4,礐,(arch.) large stone/boulder/hard,1
2590,3925,𬮤,ge2,閤,side door/variant of 閣|阁[ge2]/pavilion/cabinet...,1
2591,4411,𬮤,he2,閤,variant of 闔|阖[he2] smooth/sly slippery not...,1
2592,12004,𬮱,yin1,闉,inner gates,1


In [25]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        with uniq as (
            select id, zi, pinyin, alias, desc_en, is_active
            from t_zi_cedict c
            where not exists (
                select 1 
                from t_cedict_dedup d
                where c.id = d.id
            )

            union 

            select id, zi, pinyin, alias, desc_en, is_active
            from t_cedict_dedup d
            where is_active = 1        
        
        )
        insert into t_zi (id, zi, pinyin, alias, desc_en, is_active)
        select id, zi, pinyin, alias, desc_en, is_active from uniq
    """
    db_run_sql(sql_stmt, _conn)
    # df_12 = pd.read_sql(sql_stmt, _conn)

In [24]:
# df_12   # 11516 rows × 6 columns

,id,zi,pinyin,alias,desc_en,is_active
0,11,上,shang4,上,(bound form) up; upper,1
1,16,和,he2,和,(joining two nouns) and; together with,1
2,22,簸,bo3,簸,to winnow; to toss up and down,1
3,23,说,shuo1,說,to speak; to talk,1
4,30,绰,chao1,綽,to grab; to snatch up/variant of 焯[chao1],1
...,...,...,...,...,...,...
11511,9991,绨,ti2,綈,coarse greenish black pongee,1
11512,9992,缇,ti2,緹,orange-red silk/orange-red colored,1
11513,9996,醍,ti2,醍,essential oil of butter,1
11514,9998,𫘨,ti2,騠,spirited horse,1


### Update radicals

In [27]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        update t_zi 
        set is_radical = 1
        where exists (
            select 1 
            from t_radical_1 r 
            where r.zi = t_zi.zi
        );
    """
    db_run_sql(sql_stmt, _conn)

### Update HSK levels

In [37]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        select hsk_level,count(*) 
        from t_zi_hsk 
        group by hsk_level order by hsk_level
    """
    df_hsk = pd.read_sql(sql_stmt, _conn)

In [38]:
df_hsk

,hsk_level,count(*)
0,HSK_1-Common-01,160
1,HSK_1-Common-02,160
2,HSK_1-Common-03,160
3,HSK_1-Common-04,160
4,HSK_1-Common-05,160
5,HSK_1-Common-06,160
6,HSK_1-Common-07,160
7,HSK_1-Common-08,160
8,HSK_1-Common-09,160
9,HSK_1-Common-10,160


In [39]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        update t_zi 
        set layer = (
            select min(hsk_level)
            from t_zi_hsk k 
            where k.zi = t_zi.zi
        )
        where layer is null;
    """
    db_run_sql(sql_stmt, _conn)

In [42]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        select * from t_zi where is_active =1 and layer is not null order by layer,zi;
    """
    df_hsk = pd.read_sql(sql_stmt, _conn).fillna("")

In [43]:
df_hsk   # 3928 rows × 15 columns

,zi,desc,pinyin,nstrokes,alias,is_radical,is_zi,is_traditional,zi_en,desc_en,id,ts,uid,is_active,layer
0,一,,yi1,,一,1.0,1,,,"""one/single/a (article)/as soon as/entire; who...",11789,,,1,HSK_1-Common-01
1,丁,,ding1,,丁,,1,,,"""male adult/the 4th of the 10 Heavenly Stems 天...",3094,,,1,HSK_1-Common-01
2,七,,qi1,,七,,1,,,seven/7,8229,,,1,HSK_1-Common-01
3,万,,wan4,,萬,,1,,,ten thousand/a great number,10384,,,1,HSK_1-Common-01
4,丈,,zhang4,,丈,,1,,,"measure of length, ten Chinese feet (3.3 m)/to...",12770,,,1,HSK_1-Common-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3923,镁,,mei3,,鎂,,1,,,magnesium (chemistry),7215,,,1,HSK_3
3924,阂,,he2,,閡,,1,,,obstruct,4409,,,1,HSK_3
3925,饪,,ren4,,飪,,1,,,cooked food/to cook (until ready),8876,,,1,HSK_3
3926,馈,,kui4,,饋,,1,,,food/to make a present; to make an offering t...,6139,,,1,HSK_3


### Add Index

In [2]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        create index if not exists i_zi_zi on t_zi(zi);
        create index if not exists i_zi_id on t_zi(id);

        create index if not exists i_zi_media_zi on t_zi_media(zi);
        create index if not exists i_zi_media_id on t_zi_media(id);

        create index if not exists i_zi_part_zi on t_zi_part(zi);
        create index if not exists i_zi_part_id on t_zi_part(id);

        create index if not exists i_zi_shufa_zi on t_zi_shufa(zi);
        create index if not exists i_zi_shufa_id on t_zi_shufa(id);

        create index if not exists i_zi_text_zi on t_zi_text(zi);
        create index if not exists i_zi_text_id on t_zi_text(id);

        create index if not exists i_zi_word_zi on t_zi_word(zi);
        create index if not exists i_zi_word_id on t_zi_word(id);
    """
    db_run_sql(sql_stmt, _conn)

## Enrich with Shuowen data

see 
- Notebook: ~\zistory\shuowen\load_json.ipynb  
- DB: shuowen.duckdb
- Output:
    - zi_shuowen_picto.csv
    - zi_shuowen_composed.csv

In [4]:
df_picto = pd.read_csv(Path("C:/Users/p2p2l/projects/wgong/zistory/shuowen/zi_shuowen_picto.csv"), encoding="utf-8")

In [6]:
df_composed = pd.read_csv(Path("C:/Users/p2p2l/projects/wgong/zistory/shuowen/zi_shuowen_composed.csv"), encoding="utf-8")

In [5]:
df_picto

,zi,id_json,explanation
0,气,224,雲气也。象形。凡气之屬皆从气。
1,口,779,人所以言食也。象形。凡口之屬皆从口。
2,局,957,促也。从口在尺下，復局之。一曰博，所以行棊。象形。
3,凵,969,張口也。象形。凡凵之屬皆从凵。
4,㐁,1444,舌皃。从𧮫省。象形。
...,...,...,...
226,酉,9757,就也。八月黍成，可爲酎酒。象古文酉之形。凡酉之屬皆从酉。
227,亥,9833,荄也。十月，微陽起，接盛陰。从二，二，古文上字。一人男，一人女也。从乙，象褢子咳咳之形。《春...
228,广,5892,因广爲屋，象對剌高屋之形。凡广之屬皆从广。讀若儼然之儼。
229,亦,6582,人之臂亦也。从大，象兩亦之形。凡亦之屬皆从亦。


In [7]:
df_composed

,zi,p1,p2,id_json,explanation
0,元,一,兀,2,始也。从一从兀。
1,吏,一,史,5,治人者也。从一从史，史亦聲。
2,禮,示,豊,12,履也。所以事神致福也。从示从豊，豊亦聲。
3,祰,示,告,38,告祭也。从示从告聲。
4,祏,示,石,39,宗廟主也。《周禮》有郊、宗、石室。一曰大夫以石爲主。从示从石，石亦聲。
...,...,...,...,...,...
8525,液,水,夜,7357,𧗁也。从水夜聲。
8526,垑,土,多,9062,恃也。从土多聲。
8527,恂,心,旬,6726,信心也。从心旬聲。
8528,姬,女,𦣞,8072,黃帝居姬水，以爲姓。从女𦣞聲。


In [10]:
with DBConn(db_file="zinets.sqlite") as _conn:
    df_picto.to_sql("t_zi_shuowen_picto", con=_conn, if_exists="append", index=False)
    df_composed.to_sql("t_zi_shuowen_composed", con=_conn, if_exists="append", index=False)

### Update shuowen flags

- update t_zi (is_picto, is_composed) flags
- update t_zi_part table

In [6]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        select * from t_zi where is_active =1 --and zi=alias
        ;
    """
    df_traditional = pd.read_sql(sql_stmt, _conn)

In [7]:
df_traditional   # 8504  zi = alias   11513  total

,zi,desc,pinyin,nstrokes,alias,is_radical,is_zi,is_traditional,zi_en,desc_en,id,ts,uid,is_active,layer,is_picto,is_composed
0,上,None,shang4,None,上,NaN,1,None,None,(bound form) up; upper,11,None,None,1,HSK_1-Common-01,NaN,NaN
1,和,None,he2,None,和,NaN,1,None,None,(joining two nouns) and; together with,16,None,None,1,HSK_1-Common-06,NaN,1.0
2,簸,None,bo3,None,簸,NaN,1,None,None,to winnow; to toss up and down,22,None,None,1,HSK_2-CommonLow-7,NaN,1.0
3,说,None,shuo1,None,說,NaN,1,None,None,to speak; to talk,23,None,None,1,HSK_1-Common-09,NaN,NaN
4,绰,None,chao1,None,綽,NaN,1,None,None,to grab; to snatch up/variant of 焯[chao1],30,None,None,1,HSK_2-CommonLow-4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11508,绨,None,ti2,None,綈,NaN,1,None,None,coarse greenish black pongee,9991,None,None,1,None,NaN,NaN
11509,缇,None,ti2,None,緹,NaN,1,None,None,orange-red silk/orange-red colored,9992,None,None,1,None,NaN,NaN
11510,醍,None,ti2,None,醍,NaN,1,None,None,essential oil of butter,9996,None,None,1,None,NaN,1.0
11511,𫘨,None,ti2,None,騠,NaN,1,None,None,spirited horse,9998,None,None,1,None,NaN,NaN


In [10]:
df_traditional.columns 

Index(['zi', 'desc', 'pinyin', 'nstrokes', 'alias', 'is_radical', 'is_zi',
       'is_traditional', 'zi_en', 'desc_en', 'id', 'ts', 'uid', 'is_active',
       'layer', 'is_picto', 'is_composed'],
      dtype='object')

In [11]:
with DBConn(db_file="zinets.sqlite") as _conn:
    sql_stmt = """
        --update t_zi set alias = null where is_active =1 and zi = alias;
        update t_zi set is_traditional = 1 where is_active =1 and alias is not null;
    """
    db_run_sql(sql_stmt, _conn)